In [1]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, GlobalMaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import tensorflow as tf

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'COVID-19_Radiography_Dataset'
#valid_path = 'Datasets/test'


In [3]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
 
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [4]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [5]:
# useful for getting number of output classes
folders = glob('COVID-19_Radiography_Dataset/*')

In [6]:
folders

['COVID-19_Radiography_Dataset/COVID',
 'COVID-19_Radiography_Dataset/Normal',
 'COVID-19_Radiography_Dataset/Lung_Opacity',
 'COVID-19_Radiography_Dataset/Viral Pneumonia']

In [7]:
# our layers - you can add more if you want
x = GlobalMaxPooling2D()(vgg16.output)

In [8]:
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.3)(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [9]:

# view the structure of the model
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy', tf.keras.metrics.Recall(name='Recall'),\
                                 tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )]
)


In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

#test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 subset='training')

Found 16933 images belonging to 4 classes.


In [13]:
test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            subset='validation')

Found 4232 images belonging to 4 classes.


In [14]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_AUC', factor=0.1, patience=3, verbose=1, mode='max',
    min_delta=0.0001, cooldown=0, min_lr=0
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=3)

In [15]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[reduce_lr,early_stop]
)

Epoch 1/10
1059/1059 [==============================] - 148s 137ms/step - loss: 0.8636 - accuracy: 0.6476 - Recall: 0.5313 - Precision: 0.7257 - AUC: 0.8262 - val_loss: 0.6018 - val_accuracy: 0.7765 - val_Recall: 0.7169 - val_Precision: 0.8080 - val_AUC: 0.9291
Epoch 2/10
1059/1059 [==============================] - 145s 137ms/step - loss: 0.6230 - accuracy: 0.7583 - Recall: 0.7018 - Precision: 0.7946 - AUC: 0.9164 - val_loss: 0.5335 - val_accuracy: 0.8006 - val_Recall: 0.7606 - val_Precision: 0.8329 - val_AUC: 0.9448
Epoch 3/10
1059/1059 [==============================] - 146s 138ms/step - loss: 0.5625 - accuracy: 0.7825 - Recall: 0.7414 - Precision: 0.8139 - AUC: 0.9313 - val_loss: 0.5457 - val_accuracy: 0.7869 - val_Recall: 0.7495 - val_Precision: 0.8110 - val_AUC: 0.9462
Epoch 4/10
1059/1059 [==============================] - 146s 137ms/step - loss: 0.5355 - accuracy: 0.7915 - Recall: 0.7574 - Precision: 0.8236 - AUC: 0.9378 - val_loss: 0.5175 - val_accuracy: 0.8051 - val_Recall: 0

In [16]:
test_set.reset()
Y_pred = model.predict(test_set, verbose = 1)

y_pred = np.argmax(Y_pred, axis=1)

265/265 [==============================] - 28s 107ms/step


In [17]:
y_true = test_set.classes

In [18]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.3428638941398866


In [19]:
y_true

array([0, 0, 0, ..., 3, 3, 3], dtype=int32)